In [1]:
import os, pandas as pd, numpy as np, random,gc
import copy, cv2
pd.options.mode.chained_assignment = None
import torch, torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
# # fix path
# df_224_3d = pd.read_csv("./csv_pickle/df_224_embed.csv")
# ct_path_stack = []
# for old_path in df_224_3d['ct_path'].tolist():
#     old_ = '/ssd2/COVID2023_data_embed_npy/'
#     new_ = '/ssd8/2023COVID19/2023_covid/COVID2023_data_embed_npy/'
#     new_path = old_path.replace(old_, new_)
#     ct_path_stack.append(new_path)
# df_224_3d['ct_path'] = ct_path_stack
# df_224_3d.to_csv("./csv_pickle/df_224_embed_146server.csv", index=False, encoding='utf-8-sig')

In [4]:
# 修改init讀取全部embed的方法（記憶體無法負擔所有npy資料），改以呼叫index階段時再進行讀取
class COVID_Dataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, data_split, ct_len_s=50 , transform = None):
        if data_split == 'test':
            dataset = pd.read_pickle(csv_path)
            self.embed_info = dataset
            classes = [999]
            classes = sorted(list(classes))
            class_to_idx = {classes[i]: i for i in range(len(classes))}
            ct_path = np.unique(dataset.iloc[:, 2])
            imgs = []
            for i_scan_dir in tqdm(ct_path):
                temp_df = dataset[dataset['ct_path'] == i_scan_dir]
                imgs.append((i_scan_dir, 999))
            
        elif data_split == 'train' or 'valid': 
            df = pd.read_csv(csv_path)
            # df['embed'] = self.load_npy(df.ct_path.values.tolist(), df.embed.values.tolist())
            dataset = df[df['split'] == data_split]
            self.embed_info = dataset
            
            classes = set(dataset['label'])
            classes = sorted(list(classes))
            class_to_idx = {classes[i]: i for i in range(len(classes))}
            
            ct_path = np.unique(dataset.iloc[:, 4])
            imgs = []
            for i_scan_dir in ct_path:
                temp_df = dataset[dataset['ct_path'] == i_scan_dir]
                imgs.append((i_scan_dir, temp_df.iloc[0, 3]))
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.ct_len_s = ct_len_s
        self.imgs = imgs
        self.transform = transform
    def load_npy(self, npy_path, npy_file):
        new_npy_embed = []
        for path_, file_ in zip(npy_path, npy_file):
            new_npy_embed.append(np.load(os.path.join(path_, file_)))
        return new_npy_embed
    def __getitem__(self, index):
        img_scan_dir, label = self.imgs[index]
        
        label = self.class_to_idx[label]
        temp_df = self.embed_info[self.embed_info['ct_path'] == img_scan_dir]
        temp_df['embed'] = self.load_npy(temp_df['ct_path'].values.tolist(), temp_df['embed'].values.tolist())
        random.seed(4019)
        if len(temp_df) >= self.ct_len_s:
            temp_index = [x for x in range(len(temp_df))]
            target_index = random.sample(temp_index, k = self.ct_len_s)
            
        elif len(temp_df) < self.ct_len_s:
            target_index = [x for x in range(len(temp_df))]
            temp = random.choices(target_index, k = self.ct_len_s - len(target_index))
            target_index += temp
        
        target_index.sort()
        embed = temp_df.iloc[target_index, 1]
        img = []
        for i_embed in embed:
            img.append(i_embed)
        # img = np.expand_dims(np.array(img).reshape((1536, 8*8*self.ct_len_s)), axis=0)
        img = np.array(img)
        if len(img.shape)==4:
            img = np.array(img).reshape((1536, 8*8*self.ct_len_s))
        # else:
        #     img = img.reshape((img.shape[1],img.shape[0]))
        # img = np.concatenate([img,img,img], axis=0)
        # print(img.shape)
        return img, label

    def __len__(self):
        return len(self.imgs)

In [5]:
import torch,os
import torch.nn as nn
import timm
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self, ct_len=224, kernal_size = 3, pre_train=True):
        super(MyModel, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=100, out_channels=CONFIG.ct_len_get, kernel_size=kernal_size)
        # self.conv1d = nn.Conv1d(in_channels=224, out_channels=CONFIG.ct_len_get, kernel_size=kernal_size)
        self.backbone = timm.create_model('resnet18', pretrained=pre_train, num_classes=1)
    def forward(self, x):
        
        x = self.conv1d(x)
        
        x = torch.cat((x.unsqueeze(1), x.unsqueeze(1), x.unsqueeze(1)), dim=1)
        
        x = self.backbone(x)
        return x

In [6]:
def loss_fn(outputs, labels):
    return nn.BCEWithLogitsLoss()(outputs, labels)

def train_loop(model, optimizer, scheduler, loader):
    losses, lrs = [], []
    model.train()
    optimizer.zero_grad()
    for images, label in loader:
        out = model(images.cuda())

        loss = loss_fn(out.view(-1),label.cuda().float())
        
        losses.append(loss.item())
        step_lr = np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean()
        lrs.append(step_lr)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    return np.array(losses).mean(), np.array(lrs).mean()


def valid_loop(model, loader):
    losses, predicts = [], []
    true_y=[]
    pred_y=[]
    model.eval()
    for images, label in loader:
        with torch.no_grad():
            images = images.cuda().float()
            labels = label.cuda().float()
            out = model(images)
            loss = loss_fn(out.view(-1),labels)
        losses.append(loss.item())
        predicts.append(out.cpu())
        true_y.append(labels.cpu().numpy())
        pred_y.append(torch.sigmoid(out).cpu().numpy())
        
    true_y=np.concatenate(true_y)
    pred_y=np.concatenate(pred_y)
    
    gc.collect()
    
    true_y=np.array(true_y).reshape(-1,1)
    pred_y=np.array(pred_y).reshape(-1,1)
   
    acc_f1=f1_score(np.array(true_y),np.round(pred_y),average='macro')

    auc_roc=roc_auc_score(np.array(true_y),np.array(pred_y))
    print("acc_f1 : ",round(acc_f1,4),"  auc_roc : ",round(auc_roc,4))
    
    return np.array(losses).mean(),acc_f1,auc_roc



In [7]:
class CONFIG:
    # model_path1 = "/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/output/f1_best_model_k1_convslice_check_from144weight[384].bin"
    model_path1 = "/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/output/f1_best_model_k1_convembed_check_from144weight[384].bin"
    # model_path1 = "/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/output/f1_best_model_k1_convembed_check_from144weight[256].bin"
    pre_train = False
    N_EPOCHS = 100
    train_batch_size = 32
    valid_batch_size = 32
    ct_len_get = 100 #100
    kernal_size = 1 #1
    SEDD =42
    LR = 3e-5 #3e-5
    WEIGHT_DECAY = 1e-3 #1e-3

In [8]:
def main():
    print("==========loading model==========")
    set_seed(seed=42)
    model = MyModel(ct_len = CONFIG.ct_len_get, kernal_size=CONFIG.kernal_size, pre_train=CONFIG.pre_train).cuda()
    optimizer = AdamW(model.parameters(), lr=CONFIG.LR, weight_decay=CONFIG.WEIGHT_DECAY)


    df_path = './csv_pickle/df_224_embed_146server.csv'
    # df_path = './csv_pickle/df_224_embed_sz_384_146server.csv'
    print("==========data loader==========")
    train_ds = COVID_Dataset(csv_path = df_path, data_split = 'train', ct_len_s = CONFIG.ct_len_get, transform = None)
    valid_ds = COVID_Dataset(csv_path = df_path,data_split = 'valid', ct_len_s = CONFIG.ct_len_get, transform = None)
    
    train_loader = DataLoader(train_ds, batch_size=CONFIG.train_batch_size, num_workers=15, shuffle=True, pin_memory=True)
    valid_loader = DataLoader(valid_ds, batch_size=CONFIG.valid_batch_size, num_workers=15, shuffle=False, pin_memory=True)
    image, label = next(iter(train_loader))
    print(image.shape)

    num_train_steps = int(len(train_loader) * CONFIG.N_EPOCHS)
    num_warmup_steps = int(num_train_steps / 10)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)

    
    print("==========start train==========")
    best_epoch_f1 = 0
    for epoch in tqdm(range(CONFIG.N_EPOCHS)):
        train_loss, lrs = train_loop(model, optimizer, scheduler, train_loader)
        valid_loss,acc_f1,auc_roc = valid_loop(model, valid_loader)

        if acc_f1 > best_epoch_f1:
            print(f"Validation f1 Improved ({best_epoch_f1} ---> {acc_f1})")
            best_epoch_f1 = acc_f1
            #run.summary["Best Loss"] = best_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = CONFIG.model_path1
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved")
    torch.cuda.empty_cache()
    gc.collect()
    print(best_epoch_f1)

In [9]:
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
main()

==========loading model==========


/home/chihyi111/miniconda3/envs/swin/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


==========data loader==========
torch.Size([32, 100, 224])
==========start train==========


  1%|          | 1/100 [00:24<41:03, 24.89s/it]

acc_f1 :  0.3998   auc_roc :  0.8335
Validation f1 Improved (0 ---> 0.3998294970161978)
Model Saved


...

acc_f1 :  0.8833   auc_roc :  0.934


 99%|█████████▉| 99/100 [40:20<00:26, 26.89s/it]

acc_f1 :  0.8813   auc_roc :  0.9329


100%|██████████| 100/100 [40:42<00:00, 24.42s/it]

acc_f1 :  0.8833   auc_roc :  0.9336
0.8884825444051875


In [14]:
# mean 10
def valid_one(model, loader):
    losses, predicts = [], []
    true_y=[]
    pred_y=[]
    model.eval()
    for images, label in loader:
        with torch.no_grad():
            images = images.cuda().float()
            labels = label.cuda().float()
            out = model(images)
            loss = loss_fn(out.view(-1),labels)
        losses.append(loss.item())
        predicts.append(out.cpu())
        true_y.append(labels.cpu().numpy())
        pred_y.append(torch.sigmoid(out).cpu().numpy())
        
    true_y=np.concatenate(true_y)
    pred_y=np.concatenate(pred_y)
    
    gc.collect()
    
    true_y=np.array(true_y).reshape(-1,1)
    pred_y=np.array(pred_y).reshape(-1,1)

    return true_y,pred_y

In [15]:
pred_path = CONFIG.model_path1
model = MyModel(ct_len = CONFIG.ct_len_get, kernal_size=CONFIG.kernal_size, pre_train=CONFIG.pre_train).cuda()
model.load_state_dict(torch.load(pred_path))
model.cuda()
df_path = './csv_pickle/df_224_embed_sz_384_146server.csv'
print("==========data loader==========")
valid_ds = COVID_Dataset(csv_path = df_path,data_split = 'valid', ct_len_s = CONFIG.ct_len_get, transform = None)
valid_loader = DataLoader(valid_ds, batch_size=CONFIG.valid_batch_size, num_workers=15, shuffle=False, pin_memory=True)


==========data loader==========


In [16]:
total_pred=[]
for i in range(10):
    true_y,pred_y=valid_one(model, valid_loader)
    total_pred.append(pred_y)
# for i in range(len(total_pred)):
#     print(f1_score(np.array(true_y),np.round(total_pred[i]),average='macro'))  
tn, fp, fn, tp = confusion_matrix(np.array(true_y), np.round(np.mean(total_pred,axis=0))).ravel()
print("Mean F1-Score: {}".format(f1_score(np.array(true_y),np.round(np.mean(total_pred,axis=0)),average='macro')))
print("Negative Accuracy: {}".format(tn/(tn+fp)))
print("Positive Accuracy: {}".format(tp/(tp+fn)))

Mean F1-Score: 0.8884825444051875
Negative Accuracy: 0.9381663113006397
Positive Accuracy: 0.8297872340425532


In [13]:
# ======================================
# model: 2dcnn(efficientnet_b3a) + 1dcnn + resnet18
# LR: (0.0001, decay:0.0005)
# batch size: 8
# image size: [384x384]
# datatype: challenge data
# (weight =  best f1 socre checkpoint)
# ------------------------------------
# [conv axis=embedding]
# kernel=1; ct_len=100[pretrain=True]
# Mean F1-Score: 0.8964
# Negative Accuracy: 0.9744
# Positive Accuracy: 0.7872
# ------------------------------------
# [conv axis=embedding]
# kernel=3; ct_len=100[pretrain=True]
# Mean F1-Score: 0.8964
# Negative Accuracy: 0.9744
# Positive Accuracy: 0.7872
# ------------------------------------
# [conv axis=embedding]
# kernel=1; ct_len=100[pretrain=False]
# Mean F1-Score: 0.9221
# Negative Accuracy: 0.9637
# Positive Accuracy: 0.8680
# ------------------------------------
# [conv axis=slice]
# kernel=1; ct_len=100[pretrain=False]
# Mean F1-Score: 0.9133
# Negative Accuracy: 0.9658
# Positive Accuracy: 0.0.8425
# ------------------------------------
# [conv axis=embedding]
# kernel=3; ct_len=100[pretrain=False]
# Mean F1-Score: 0.9160
# Negative Accuracy: 0.9744
# Positive Accuracy: 0.8340
# ------------------------------------
# [conv axis=embedding]
# kernel=7; ct_len=100[pretrain=False]
# Mean F1-Score: 0.9150
# Negative Accuracy: 0.9658
# Positive Accuracy: 0.8468
# ------------------------------------
# [conv axis=embedding]
# kernel=1; ct_len=200[pretrain=False]
# Mean F1-Score: 0.9076
# Negative Accuracy: 0.9701
# Positive Accuracy: 0.8212
# ------------------------------------
# [conv axis=embedding]
# kernel=1; ct_len=224[pretrain=False]
# Mean F1-Score: 0.9015
# Negative Accuracy: 0.9616
# Positive Accuracy: 0.8212